This notebook is focused on the [netCDF](https://www.unidata.ucar.edu/software/netcdf/) and how to read/write data from/in this format.    
The first module to work with netCDF was https://github.com/JuliaGeo/NetCDF.jl, but recently [@abarth](https://github.com/Alexander-Barth) developed another module: [NCDatasets](https://github.com/Alexander-Barth/NCDatasets.jl). We will use the later in this notebook.

In [39]:
using NCDatasets
Pkg.status("NCDatasets")

INFO: Recompiling stale cache file /home/ctroupin/.julia/v0.6/lib/v0.6/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/ctroupin/.julia/v0.6/lib/v0.6/NCDatasets.ji for module NCDatasets.


 - NCDatasets                    0.0.9


# Write a netCDF
Let's start with a very simple example: a temperature time series (1-dimensional dataset), corresponding to the mean temperature in Uccle, Belgium (source: [IRM](https://www.meteo.be/meteo/view/fr/360955-Normales+mensuelles.html#ppt_5238240)).

In [49]:
temperatureUccle = [3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18., 14.9, 11.1, 6.8, 3.9]

12-element Array{Float64,1}:
  3.3
  3.7
  6.8
  9.8
 13.6
 16.2
 18.4
 18.0
 14.9
 11.1
  6.8
  3.9

First, let's create the file (clobber mode indcated by "c"):

In [53]:
ds = Dataset("temperature_series.nc", "c");

Then we create a time dimension. Its size should correspond to that of our temperature vector.

In [54]:
defDim(ds, "time", length(temperatureUccle))

0

Now we can also create the *temperature* variable, which has *time* as dimension.

In [55]:
temperature = defVar(ds, "temperature", Float64, ("time",));

We will this variable with the values previously defined:

In [56]:
temperature[:] = temperatureUccle;

and don't forget to close the `ds`:

In [57]:
close(ds)

If you have [`ncdump`] installed on your machine, you can easily check the file content.

In [58]:
run(`ncdump temperature_series.nc`)

netcdf temperature_series {
dimensions:
	time = 12 ;
variables:
	double temperature(time) ;
data:

 temperature = 3.3, 3.7, 6.8, 9.8, 13.6, 16.2, 18.4, 18, 14.9, 11.1, 6.8, 3.9 ;
}
